In [1]:
import cv2
import dlib
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os

In [2]:
# Initialize dlib's face detector and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r"C:\Users\Bimlendra\Downloads\shape_predictor_68_face_landmarks.dat")


In [3]:
# Custom Dataset for microexpression videos
class MicroexpressionDataset(Dataset):
    def __init__(self, dataset_dir, transform=None):
        self.dataset_dir = dataset_dir
        self.transform = transform
        self.data = []
        self.labels = []
        
        # Assume dataset_dir has subfolders 'neutral' and 'deceptive'
        for label, class_name in enumerate(['neutral', 'deceptive']):
            class_dir = os.path.join(dataset_dir, class_name)
            for file_name in os.listdir(class_dir):
                if file_name.endswith(('.mp4', '.avi')):
                    self.data.append(os.path.join(class_dir, file_name))
                    self.labels.append(label)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        video_path = self.data[idx]
        label = self.labels[idx]
        
        # Extract first frame (simplified; extend to process multiple frames)
        cap = cv2.VideoCapture(video_path)
        ret, frame = cap.read()
        cap.release()
        if not ret:
            return None, None
        
        # Detect face and crop
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)
        if len(faces) == 0:
            return None, None
        
        face = faces[0]
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        face_crop = gray[y:y+h, x:x+w]
        face_crop = cv2.resize(face_crop, (64, 64))  # Resize to 64x64
        
        # Convert to tensor
        face_crop = face_crop.astype(np.float32) / 255.0
        face_crop = torch.tensor(face_crop).unsqueeze(0)  # Shape: [1, 64, 64]
        
        if self.transform:
            face_crop = self.transform(face_crop)
        
        return face_crop, label

In [4]:
# Simple CNN Model
class MicroexpressionCNN(nn.Module):
    def __init__(self):
        super(MicroexpressionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: neutral, deceptive
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
# Training function
def train_model(dataset_dir, model_path="cnn_model.pth"):
    # Data preprocessing
    transform = transforms.Compose([transforms.Normalize((0.5,), (0.5,))])
    dataset = MicroexpressionDataset(dataset_dir, transform=transform)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)
    
    # Initialize model, loss, and optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MicroexpressionCNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Training loop
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(dataloader):
            if inputs is None or labels is None:
                continue
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 10 == 9:
                print(f"Epoch {epoch+1}, Batch {i+1}, Loss: {running_loss/10:.4f}")
                running_loss = 0.0
    
    # Save the model
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")

# Example usage
if __name__ == "__main__":
    dataset_dir = "path_to_dataset"  # Replace with your dataset path
    train_model(dataset_dir)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'path_to_dataset\\neutral'